In [4]:
print("Hello World")

Hello World


In [14]:
import time
import hashlib
import os

session = os.urandom(32)
timestamp = str(time.time()).encode()
padded_timestamp = timestamp.ljust(32, b'\x00')

session_key = session + padded_timestamp

# print(session_key)
# print(len(session_key))

rec_time = session_key[32:]
decoded_time = rec_time.decode().rstrip('\x00')
timestamp_rec = float(decoded_time)
print(timestamp_rec)

now = time.time()
diff_time = now - timestamp_rec + 12

if abs(diff_time) <=10:
    print("Time stamp is within the alloted time")
else:
    print("Incorrect time stamp, TIME OUT or ATTACK. NICE TRY HACKER!!!")


1745288806.568347
Incorrect time stamp, TIME OUT or ATTACK. NICE TRY HACKER!!!


In [ ]:
import os, time
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding as sym_padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.backends import default_backend

def aes_env():
    session = os.urandom(32)
    time_stmp = str(time.time()).encode()
    padded_timestamp = time_stmp.ljust(32, b'\x00')
    session_key = session + padded_timestamp
    iv = os.urandom(16)
    aes_key = os.urandom(32)
    
    '''
    Notes from the crypto homework, no need to reinvent the wheel
        sym_key was given directly to the function in the crypto file
        param sym_key: the symmetric secret key (byte string)
            I believe this maybe one of the areas where we combine multiple encryption types,
            like the envelope method but with CBC mode. So generate data with sym_key, then
            encrypt with RSA key then send data.

        param iv: initial value used during encryption (byte string)

        Data will probably be session key, this is the infor that really important
    '''
    #Encrypting using AESCBS Mode
    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    #Padding
    padder = sym_padding.PKCS7(128).padder()
    padded_data = padder.update(session_key) + padder.finalize()
    cipher_text = encryptor.update(padded_data) + encryptor.finalize()

    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()

    #Encrypt AES KEY
    # public_key = None # Retrieve key from file
    enc_key = public_key.encrypt(
        aes_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    env_message = enc_key + iv + cipher_text

    return env_message, private_key

print(aes_env())

b'\xc5\xf0u\x8cC\x8e\x87\xe7\x08s1J\x08\x12\x19O\xd9\xf2t\x93\x94\xddff\xc8\x8a\xee*^j\xbd\xed\xdaS^H\xc6o\xd6\x19K[\x13\xaa}uhTP\x14\x8b\xad\xf6\xf9{\xd1P\xb56Q\xac\xc3yzqum\xf52\xac\xaaO\xa4\xe7:Uv\xd2\xecuJkS\x90a\xc8m\xc2\x82\x87\xaf\x83y\x92\xfe\xd06\xcc*rE\r}\xb5\xed\x82\xf2\xd7C\xd1\xc4\x05f\xfb\x03\x9f\xb2\x9f`\xf3\xb5t\xe3\x95\x86\xc0g\n\xd2ET\x91\xcf8\x04\xfb_S\x8b(\xc7G\x1c\x95\x9fD_DL\x88\x89\xb1s\x17\xaf\xcd!\x89\xd5\x9ag\xf62\x9e\xaf\xb7\xf3\xf5kw\xbd5\xfbv\xd0\xb8\xe8o\xdcY\xb4b\xcb\x978S\x92\xb1\x06=\xe6\xc6H\xaa\xcf\x8e_\x98\xb8\x10\xa0\xf1Y\xa1\x9bN\xac\x84\x93`\x08\xde=\xa9\x83\x057Y&\x1d\x8c=\xf9\x9f\xf9x_\x8b\\\x08\x0b\xbe\x93t\xca\xb7\xfa\xd8\xe7\xfa\xb4j\x14\xef\x18\x98\xe8\x0c\xd2\x96\xfc&\x00\xc9\xae\xfd\xed\xbd\x00<~\xb8\x9d\x82\xab\xb8\xb2\xe4+U;\xb2\xcb\xd0\xb3\x82\x91\xd3\xa7\t\xef\xf4J\xd4\x1c\xb0\x8d@Y\xc6@\xc3j\xc0G\xed\xd9\xc6g\x96zd(\xd2\\\xc9\xaa\xcd!\xd3r\xfa\xa6\xe4}\x05\xf1\xb3\xe2Y$\x15\xa7&\n\xf3\\\x98\x19-y\xfb\xf8\xfa\xd1\x05\xf0\x8d\x99}\xca?\

## Final Message size
- enc_key = 256 bytes
- iv = 16 bytes
- cipher_text = 80 bytes
    - I can also set the code so the last of the message is the cipher text.

In [ ]:
import os, time
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding as sym_padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.backends import default_backend

def dec_env(message):
    enc_key, iv, cipher_text = message